In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: US-Gun-Violence.csv     


In [ ]:
#import necessary packages
!pip install geopandas
!pip install geopy

In [6]:
import csv
import geopy
import geopandas
import pandas as pd
import plotly.express as px

In [4]:
gun_report= pd.read_csv("US-Gun-Violence.csv")
gun_report.head()

,incident_id,incident_date,state,city_or_county,address,killed,injured
0,2128682,September 29 2021,Illinois,Chicago,800 block of N Milwaukee Ave,0,5.0
1,2127374,September 28 2021,Illinois,Chicago,65th St and Calumet Ave,1,3.0
2,2128023,September 28 2021,Minnesota,Minneapolis,2700 block of Bloomington Ave,1,3.0
3,2126747,September 27 2021,New York,New York (Manhattan),3964 10th Ave,0,5.0
4,2126450,September 27 2021,Illinois,Chicago,826 N Ridgeway Ave,1,3.0


In [7]:
# Get a dictionary for state abbreviations
states = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [8]:
gun_report['state code'] = gun_report['state'].apply(lambda x : states[x])

In [9]:
gun_report.head()
#Now the column "state code" is available for address encoding

,incident_id,incident_date,state,city_or_county,address,killed,injured,state code
0,2128682,September 29 2021,Illinois,Chicago,800 block of N Milwaukee Ave,0,5.0,IL
1,2127374,September 28 2021,Illinois,Chicago,65th St and Calumet Ave,1,3.0,IL
2,2128023,September 28 2021,Minnesota,Minneapolis,2700 block of Bloomington Ave,1,3.0,MN
3,2126747,September 27 2021,New York,New York (Manhattan),3964 10th Ave,0,5.0,NY
4,2126450,September 27 2021,Illinois,Chicago,826 N Ridgeway Ave,1,3.0,IL


In [14]:
#Group killed victims by state
state_killed = gun_report.groupby('state code')['killed'].sum().sort_values(ascending = False).reset_index()
state_killed.head()

,state code,killed
0,TX,355
1,CA,344
2,FL,273
3,IL,235
4,OH,142


In [15]:
fig = px.choropleth(state_killed, locations='state code', locationmode="USA-states", color='killed', scope="usa", color_continuous_scale="Viridis")
fig.show()
#Get a map of victim killed in gun violence by state

In [13]:
#State-wise injuries
state_injuries = gun_report.groupby('state code')['injured'].sum().sort_values(ascending = False).reset_index()
state_injuries.head()

,state code,injured
0,IL,1484.0
1,CA,1235.0
2,FL,892.0
3,TX,840.0
4,NY,653.0


In [17]:
#Map of injured gun violence victim
fig_ = px.choropleth(state_injuries, locations='state code', locationmode="USA-states", color='injured', scope="usa", color_continuous_scale="Viridis")
fig_.show()

In [30]:

#Graph with raw data
fig1 = px.histogram(gun_report,x='state',color='state',template='plotly_dark')
fig1.show()


In [32]:
gun_report['full address'] = gun_report['city_or_county']+','+gun_report['state code']

In [46]:
fig2 = px.histogram(gun_report,x='city_or_county',color='city_or_county',template='plotly_dark')
fig2.show()

In [60]:
from geopy.geocoders import Nominatim
import numpy as np

geolocator = Nominatim(user_agent='myapplication')
gun_report['full address'] = gun_report['city_or_county']+' '+gun_report['state']
gun_report['lat'] = 0
gun_report['lon'] = 0

In [ ]:
#Due to service time limit, we are only mapping the most recent 200 cases
lats = []
lons = []
for index, row in gun_report.iterrows():
  loc = geolocator.geocode(row['full address'])
  if loc is not None and index <= 200:
    row['lat'] += geolocator.geocode(row['full address']).latitude
    lats.append(row['lat'])
    row['lon'] += geolocator.geocode(row['full address']).longitude
    lons.append(row['lon'])
    print(index)
  if index > 200:
    break
    print("Done")


In [76]:
#The addresses are incomplete, so we can do the hotspot analysis on 
# cities/counties by getting their coordinates.
lats1, lons1 = np.array(lats), np.array(lons)
lats1 = np.reshape(lats1,(-1,1))
lons1 = np.reshape(lons1,(-1,1))
locs = np.concatenate((lats1, lons1), axis = 1)
print(locs)


[[  41.8755616   -87.6244212 ]
 [  41.8755616   -87.6244212 ]
 [  44.9772995   -93.2654692 ]
 [  40.7587979   -73.9623427 ]
 [  41.8755616   -87.6244212 ]
 [  41.8755616   -87.6244212 ]
 [  37.8044557  -122.2713563 ]
 [  47.40033525 -122.29682293]
 [  43.1009031   -75.2326641 ]
 [  35.4428864   -84.5930113 ]
 [  35.0457219   -85.3094883 ]
 [  33.9533546  -117.3961623 ]
 [  33.5598586   -81.721952  ]
 [  34.0536909  -118.242766  ]
 [  35.042036    -89.6645266 ]
 [  39.9527237   -75.1635262 ]
 [  35.0525759   -78.878292  ]
 [  38.8950368   -77.0365427 ]
 [  39.9527237   -75.1635262 ]
 [  38.6264256   -90.1995853 ]
 [  32.6967679   -80.8511281 ]
 [  25.7741728   -80.19362   ]
 [ -23.6475942   -46.5101346 ]
 [  37.8044557  -122.2713563 ]
 [  41.8755616   -87.6244212 ]
 [  43.157285    -77.615214  ]
 [  40.8466508   -73.8785937 ]
 [  40.125222    -87.6304614 ]
 [  30.5335302   -92.081509  ]
 [  33.4709714   -81.9748429 ]
 [  39.2908816   -76.610759  ]
 [  41.5051613   -81.6934446 ]
 [  43.0

In [77]:
import folium
from folium import plugins
from folium.plugins import HeatMap
map_1 = folium.Map(location=[35, -95], tiles='cartodbpositron', zoom_start=4)
HeatMap(data=locs, radius=0).add_to(map_1)
map_1

In [78]:
f = folium.Figure(width=1200, height=750)
map = folium.Map(location = [25, -95], 
                      zoom_start = 4).add_to(f)
price=folium.map.FeatureGroup()
price=plugins.MarkerCluster().add_to(map)
for loc,label in zip(locs,gun_report.incident_date):
    folium.Marker(location=loc,icon=None,popup=label).add_to(price)
map.add_child(price)
map